In [ ]:
import pygame
import random
import math

pygame.init()

# --- helper to load word lists from text files ---
def load_word_list(filename):
    words = []
    try:
        with open(filename, "r", encoding="utf-8") as f:
            for line in f:
                w = line.strip().lower()
                if w:
                    words.append(w)
    except FileNotFoundError:
        print(f"Warning: {filename} not found, using empty list.")
    return words

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 200)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 28)
BIG = pygame.font.SysFont("arial", 56)

clock = pygame.time.Clock()

# --- คำถาม (categories) ---  (NOW LOADED FROM FILES)
questions = {
    "Name a fruit": load_word_list("fruits.txt"),
    "Name a country": load_word_list("countries.txt"),
    "Name an animal": load_word_list("animals.txt")
}


# --- บล็อก ---
block_width = 50
block_height = 30


class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


# --- Water class (easing + nicer waves) ---
class Water:
    def __init__(self, screen_h, start_level=None, anim_dur=0.9, color=BLUE):
        self.screen_h = screen_h
        self.level = float(start_level if start_level is not None else screen_h)  # world y
        # animation fields for smooth rising
        self.start_y = self.level
        self.target_y = self.level
        self.anim_time = 0.0
        self.anim_dur = anim_dur
        self.animating = False
        self.color = color

        # store correct word lengths only
        self.word_lengths = []

    def rise(self, amount_px):
        """Start a smooth rise by amount_px (pixels). Uses easing animation."""
        if amount_px <= 0:
            return
        # allow stacking rises while animating: start from current level
        self.start_y = float(self.level)
        # ❌ เอา max(0.0, ...) ออก เพื่อให้ขึ้นเกินขอบบนได้
        self.target_y = self.start_y - float(amount_px)
        self.anim_time = 0.0
        self.animating = True


    def update(self, dt):
        """Update easing animation (smoothstep)."""
        if not self.animating:
            return
        self.anim_time += dt
        t = min(1.0, self.anim_time / self.anim_dur)
        # smoothstep easing
        t_ease = t * t * (3 - 2 * t)
        self.level = self.start_y + (self.target_y - self.start_y) * t_ease
        if t >= 1.0:
            self.level = self.target_y
            self.animating = False

    def draw(self, surface, camera_y=0):
        """Draw water with two-layer waves & subtle highlight."""
        screen_y = int(self.level - camera_y)
        height = max(0, self.screen_h - screen_y)
        if height <= 0:
            return

        # gradient surface
        grad = pygame.Surface((WIDTH, height), pygame.SRCALPHA)
        top_col = (25, 60, 160, 220)
        bottom_col = (60, 150, 230, 200)
        for i in range(height):
            r = int(top_col[0] + (bottom_col[0] - top_col[0]) * (i / max(1, height - 1)))
            g = int(top_col[1] + (bottom_col[1] - top_col[1]) * (i / max(1, height - 1)))
            b = int(top_col[2] + (bottom_col[2] - top_col[2]) * (i / max(1, height - 1)))
            a = int(top_col[3] + (bottom_col[3] - top_col[3]) * (i / max(1, height - 1)))
            pygame.draw.line(grad, (r, g, b, a), (0, i), (WIDTH, i))
        surface.blit(grad, (0, screen_y))

        # waves (back + front)
        t = pygame.time.get_ticks() / 1000.0
        points_back = []
        amp_back = 8
        wave_len_back = 240
        speed_back = 0.6
        for x in range(0, WIDTH + 8, 8):
            y = screen_y + math.sin((x / wave_len_back) + t * speed_back) * amp_back
            points_back.append((x, y))
        if len(points_back) > 1:
            pygame.draw.lines(surface, (20, 60, 120), False, points_back, 2)

        points_front = []
        amp_front = 4
        wave_len_front = 100
        speed_front = 1.4
        for x in range(0, WIDTH + 6, 6):
            y = screen_y + math.sin((x / wave_len_front) + t * speed_front) * amp_front - 2
            points_front.append((x, y))
        if len(points_front) > 1:
            pygame.draw.lines(surface, (180, 220, 255), False, points_front, 2)

        # highlight band (soft)
        hl_points = []
        for x in range(0, WIDTH + 6, 12):
            y = screen_y + math.sin((x / 60.0) + t * 2.0) * 2.0 - 6
            hl_points.append((x, y))
        if len(hl_points) > 1:
            s = pygame.Surface((WIDTH, 24), pygame.SRCALPHA)
            pygame.draw.lines(s, (255, 255, 255, 40), False, [(p[0], p[1] - screen_y + 6) for p in hl_points], 3)
            surface.blit(s, (0, screen_y - 12))

    def is_animating(self):
        return self.animating

    def reset(self, start_level=None):
        start = start_level if start_level is not None else self.screen_h
        self.level = float(start)
        self.start_y = self.level
        self.target_y = self.level
        self.anim_time = 0.0
        self.animating = False
        self.word_lengths = []


# --- helpers ---
blocks = []


def add_word_blocks(word):
    word = word.strip()
    if word == "":
        return
    if len(blocks) == 0:
        base_top_y = HEIGHT - 130 - block_height
    else:
        current_top = min(b.y for b in blocks)
        base_top_y = current_top - block_height
    x_center = WIDTH // 2 - block_width // 2
    for i, ch in enumerate(word):
        y = base_top_y - i * block_height
        blocks.append(Block(x_center, y, ch))


def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))


# --- main ---
def main():
    running = True
    input_text = ""

    # choose ONE category for the whole run
    chosen_category = random.choice(list(questions.keys()))
    current_question = questions[chosen_category]

    camera_y = 0
    camera_target_y = 0

    water = Water(screen_h=HEIGHT, start_level=HEIGHT - 100, anim_dur=0.9)
    WATER_RISE_PIXELS_PER_LETTER = block_height

    round_number = 1
    PERCENT_START = 0.50
    PERCENT_STEP = 0.10

    score = 0
    game_over = False

    submitted_words = set()  # case-insensitive stored as lower()

    info_text = ""
    info_time = 0.0
    INFO_DURATION = 2.0

    while running:
        dt = clock.tick(60) / 1000.0

        if info_text and (pygame.time.get_ticks() / 1000.0 - info_time) > INFO_DURATION:
            info_text = ""

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        submitted_word = input_text.strip().lower()
                        if submitted_word != "":
                            # duplicate -> treat like wrong: no block add, but water rises
                            if submitted_word in submitted_words:
                                info_text = "Already used!"
                                info_time = pygame.time.get_ticks() / 1000.0

                                # compute rise just like wrong (do NOT append to correct-word list)
                                percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                # avg of correct words if exist, else fallback to length of duplicated word
                                if len(water.word_lengths) > 0:
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                else:
                                    avg_len = len(submitted_word)
                                water_rise_letters = avg_len * percent
                                if water_rise_letters > 0:
                                    rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                    water.rise(rise_px)
                                round_number += 1

                            else:
                                # not duplicate -> check correctness
                                if submitted_word in current_question:
                                    submitted_words.add(submitted_word)
                                    add_word_blocks(submitted_word)
                                    score += len(submitted_word) * 10

                                    # append only correct words to avg list
                                    water.word_lengths.append(len(submitted_word))

                                    percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                    avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                    water_rise_letters = avg_len * percent
                                    if water_rise_letters > 0:
                                        rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                        water.rise(rise_px)

                                    round_number += 1

                                    info_text = "Correct!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    # IMPORTANT: do NOT change category mid-run
                                else:
                                    # wrong (not duplicate): no append to avg
                                    info_text = "Wrong!"
                                    info_time = pygame.time.get_ticks() / 1000.0

                                    if round_number == 1:
                                        # first-round wrong = instant death
                                        game_over = True
                                    else:
                                        # compute rise using avg if exists otherwise fallback to wrong len
                                        percent = PERCENT_START + PERCENT_STEP * (round_number - 1)
                                        if len(water.word_lengths) > 0:
                                            avg_len = sum(water.word_lengths) / len(water.word_lengths)
                                        else:
                                            avg_len = len(submitted_word)
                                        water_rise_letters = avg_len * percent
                                        if water_rise_letters > 0:
                                            rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                                            water.rise(rise_px)
                                        round_number += 1

                        input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        if event.unicode and event.unicode.isprintable():
                            input_text += event.unicode
                else:
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        chosen_category = random.choice(list(questions.keys()))
                        current_question = questions[chosen_category]
                        water.reset(start_level=HEIGHT - 100)
                        round_number = 1
                        score = 0
                        camera_y = 0
                        camera_target_y = 0
                        game_over = False
                        submitted_words.clear()
                        info_text = ""
                        info_time = 0.0

        # update water animation
        water.update(dt)

        # camera follow
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            tower_top_in_view = top_block_y - camera_y
            tower_bottom_in_view = bottom_block_y - camera_y

            if tower_top_in_view < HEIGHT / 4:
                camera_target_y = top_block_y - HEIGHT / 4
            elif tower_bottom_in_view > HEIGHT * 3 / 4:
                camera_target_y = bottom_block_y - HEIGHT * 3 / 4
        else:
            camera_target_y = 0

        camera_y += (camera_target_y - camera_y) * 0.12

        # GAME OVER CHECK: immediate when water reaches top block (no need to wait animation)
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if water.level <= top_block_y:
                game_over = True

        # draw scene
        screen.fill(BLACK)
        for block in blocks:
            block.draw(screen, camera_y)
        water.draw(screen, camera_y)

        # UI
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Category: {chosen_category}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 220, 20)
        draw_text(screen, f"Round: {round_number}", FONT, LIGHT_BLUE, WIDTH - 220, 50)

        avg_display = 0.0
        if len(water.word_lengths) > 0:
            avg_display = sum(water.word_lengths) / len(water.word_lengths)
        draw_text(screen, f"AvgLen: {avg_display:.2f}", FONT, LIGHT_BLUE, WIDTH - 220, 80)

        # info text
        if info_text:
            color = YELLOW
            if info_text.lower().startswith("wrong") or info_text.lower().startswith("already"):
                color = RED
            draw_text(screen, info_text, FONT, color, 20, 130)

        # overlay when game over
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 140, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-11-13 21:17:40.520 Python[38528:13166967] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 